In [ ]:
# make figures interactive:
%matplotlib notebook 
# import library containing image reduction tools for this class
from image_reduction_tools import *

In [ ]:
# first lets look at something pretty just to get the hang of how to display fits images in python
files=glob.glob('../../data/photo/M1/p*f1.fits')
data=read_raw_image(files[1])
plt.figure(figsize=(8,8))
# experiment with different scalings:
# linear
#plt.imshow(data,cmap=cm.gray,norm=None,origin='lower',aspect='auto')
# log
plt.imshow(data,cmap=cm.gray,norm=LogNorm(),origin='lower',aspect='auto')
# the origin='lower' option is to have the right orientation.
# it may be different with the most recent camera settings. TBD
plt.show()
# Noice

In [ ]:
# Why is this? 
# lets have a look at the dynamics of the image!
bins=np.linspace(0.,65000.,200)
plt.figure(figsize=(8,5))
h,bins=np.histogram(data.flatten(),bins)
plt.plot(bins[:-1],h)
plt.yscale('log')
#plt.xscale('log')
plt.title('Pixels histogram')
plt.xlabel('ADU')
plt.ylabel('counts')
plt.grid(True)
plt.show()
# discuss the different regimes in the hitogram: 
# 1 - what is the large population of darkest pixels?
# 2 - what is the small population of brightest pixels?
# 3 - Where in this plot lives the nebulosity of interest?

In [ ]:
# HEADERS! Getting information about the exposures is a vital skill!
# have a look at the header of the files for M1
files=glob.glob('../../data/photo/M1/p*f1.fits')
print_header(files[0],l=1)
#for file in files:
#    print(file)
#    print_header(file,keywords_list=['OBJECT'],l=0) # NOTE: relevant keyword to print may change depending on camera software setup


In [ ]:
# Now let's start a proper image reduction.
# We will need the master bias and the master flat first.
# Let's make a bias
# where is the data?
files=glob.glob('../../data/photo/offsets/p*.fits')
data=read_raw_image(files[4])
# what does it look like?
plt.figure(figsize=(8,8))
# setup the cuts of the image
vmin,vmax=get_scaling(data)
plt.imshow(data,cmap=cm.gray,vmin=vmin,vmax=vmax)
plt.colorbar()
plt.show()

# What are all these white dots? do they appear in all images?

In [ ]:
# OK let's try to make a master_bias, that requires medianning the 5 offset files
# first lets get some stats for the images:
files=glob.glob('../../data/photo/offsets/p*.fits')

print('filename,','Npix','avg,','rms,','min,','max')
for file in files:
    stats=get_stats(file)
    print(stats)
    
# whats' strange with these numbers?
# Do the stats of the pixels follow a classical gaussian distribution law?

In [ ]:
##############
# MASTER_BIAS
##############
# NOW compute the median of the 5 bias and store that into master_bias
files=glob.glob('../../data/photo/offsets/p*.fits')
nfiles=len(files)
data,header_ori=read_raw_image(files[0],get_header=1)
nx,ny=np.shape(data)
datastore=np.full((nx,ny,nfiles),0.)
# store individual images into an array
for i in range(nfiles):
    datastore[:,:,i]=read_raw_image(files[i])
# compute median of nfiles images
result=np.median(datastore,axis=2)
target_file='../../masters/photo/master_bias.fits'
# update the OBJECT field of header
header=header_ori
header['OBJECT']='MASTER_BIAS'
# write to file
write_fits_image(target_file,result,header)


In [ ]:
# Whats the new stats? Any improvement?
master_bias_file='../../masters/photo/master_bias.fits'
master_bias,header1=read_raw_image(master_bias_file,get_header=1)
print('MASTER_BIAS')
print(get_stats(master_bias_file))
print('individual biases')
for file in files:
    print(get_stats(file))

# yay stats of master_bias look much better!!

    
#How does it look?
plt.figure(figsize=(6,6))
plt.imshow(master_bias,cmap=cm.gray)
plt.show()


In [ ]:
# Now lets make a flat!
files=glob.glob('../../data/photo/flats/B/p*.fits')
nfiles=len(files)
# what does it look like?
data=read_raw_image(files[0])
plot_image(data)
# discuss the various structures we can see


In [ ]:
# check the stats of the flats:
files=glob.glob('../../data/photo/flats/B/p*.fits')
for file in files:
    print(get_stats(file))
# What differences with the bias stats?

In [ ]:
# plot the central line of each flat:
files=glob.glob('../../data/photo/flats/V/p*.fits')
#files=glob.glob('../data/photo/offsets/p*.fits')
plt.figure(figsize=(8,5))
iline=511
for file in files:
    data=read_raw_image(file)
    plt.plot(data[iline,:],label=file)
plt.legend()
plt.show()

# what do you notice?

In [ ]:
# Remove the master_bias from each flat, then compute the normalized flats, where each flat is divided by its median:
files=glob.glob('../../data/photo/flats/V/p*.fits')
for file in files:
    data,header=read_raw_image(file,get_header=1)
    print(np.median(data))
    bias=read_raw_image('../../masters/photo/master_bias.fits')
    data=data-bias # Bias subtraction!
    data=data/np.median(data) # rescaling so that median(data)=1
    newfile=file+'.norm'
    write_fits_image(newfile,data,header)

In [ ]:
# now plot these normalized flats:
# plot the central line of each flat:
files=glob.glob('../../data/photo/flats/V/p*.fits.norm')
plt.figure(figsize=(8,5))
iline=511
for file in files:
    data=read_raw_image(file)
    plt.plot(data[iline,:],label=file)
plt.legend()
plt.show()
    

In [ ]:
# then median the normalized flats to get the master_flat in the considered filter
nfiles=len(files)
data,header_ori=read_raw_image(files[0],get_header=1)
nx,ny=np.shape(data)
datastore=np.full((nx,ny,nfiles),0.)
# store individual images into an array
for i in range(nfiles):
    datastore[:,:,i]=read_raw_image(files[i])
    
# compute median of nfiles images
result=np.median(datastore,axis=2)
target_file='../../masters/photo/master_flat_V.fits'
# update the OBJECT field of header (its not mandatory but will help when
# we try to make sense of / analyse the data / its very late / we did not sleep much
# / and everybody is tired)
header=header_ori
header['OBJECT']='MASTER_FLAT'
# write to file
write_fits_image(target_file,result,header)

In [ ]:
# now plot normed flats as above and overplot master
files=glob.glob('../../data/photo/flats/V/p*.fits.norm')
plt.figure(figsize=(8,5))
iline=511
offset=0
for file in files:
    data=read_raw_image(file)
    plt.plot(data[iline,:]+offset,label=file)
    offset=offset#+0.05

data=read_raw_image('../../masters/photo/master_flat_V.fits')
plt.plot(data[iline,:]+offset,label='master')
plt.legend()
plt.show()

In [ ]:
# Now reduce the observed images:
# 1=> subtract bias
# 2=> divide by flat WITH THE CORRESPONDING FILTER!!
files=glob.glob('../../data/photo/M1/p439*.fits')
filter_target='V Cousins'
nfiles=len(files)
for file in files:
    data,header=read_raw_image(file,get_header=1)
    filter_obs=header['FLTRNM']
    if(filter_obs.find(filter_target)>=0): # this checks that the filters match
        print('reducing ',file, filter_obs)
        bias=read_raw_image('../../masters/photo/master_bias.fits')
        master_flat=read_raw_image('../../masters/photo/master_flat_V.fits')
        # THE IMPORTANT, KEY PART!!!
        ##########################
        reduced=(data-bias)/master_flat
        ##########################
        reduced_file=file+'.reduced'
        write_fits_image(reduced_file,reduced,header)

# do this for each filter or automatize

In [ ]:
# Now redo the steps for the Halpha, OIII and B filters of M1
# OR DO a completely different objct, e.g. NGC2392
# => generate master_flat for each filter
# => reduce all Halpha, OIII and B images of M1 (i.e. subtract master_bias, divide by master_flat)
# Then lets RGB compose M1